In [ ]:
from pynq.overlays.base import BaseOverlay
from pynq.lib.arduino.arduino_io import Arduino_IO
from pynq.lib.video import *
from pynq.lib import Pmod_IO
import cv2
import time
import numpy as np

In [ ]:
base = BaseOverlay("base.bit")

In [ ]:
class DinoAgent:
    def __init__(self):
        self.space = Arduino_IO(base.ARDUINO, 0, 'out')
        self.space.write(0) 
        frame_out_w = 1920
        frame_out_h = 1080
        Mode_in = VideoMode(frame_out_w, frame_out_h,24)
        Mode_out = VideoMode(frame_out_w, frame_out_h, 24)
        self.hdmi_in = base.video.hdmi_in
        self.hdmi_in.configure()
        print(self.hdmi_in.mode)
        self.hdmi_in.start()
        print("HDMI In Initialized")

        self.hdmi_out = base.video.hdmi_out
        self.hdmi_out.configure(Mode_out)
        self.hdmi_out.start()
        print(self.hdmi_in.mode)
        print(self.hdmi_out.mode)
        print("HDMI Out Initialized")
        
        self.hdmi_in.tie(self.hdmi_out)
        self.flag=1
        print("Game start")
        self.jump()
        self.flag=0
        self.bitwise_mode = 0

    def jump(self):
        if self.flag == 0:
            self.space.write(1)
            time.sleep(0.1)
        else:
            self.space.write(0)
            
    def video_pipeline(self):
        try:
            while True:
                try:
                    inframe = self.hdmi_in.readframe()
                    inframe_gray = cv2.cvtColor(inframe, cv2.COLOR_BGR2GRAY)
                    ret, thr = cv2.threshold(inframe_gray, 150, 255, cv2.THRESH_BINARY)
                    thr = thr[400:500,150:250] # frame location to identify obstacles
                    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (5, 5))
                    thr1 = cv2.morphologyEx(thr, cv2.MORPH_CLOSE, kernel)
                    self.judge(thr1)
                    inframe.freebuffer()
                except KeyboardInterrupt:
                    self.hdmi_out.close()
                    self.hdmi_in.close()
                    print('close pipeline')
                    raise
                except ValueError:
                    self.hdmi_out.close()
                    self.hdmi_in.close()
                    print('val err close pipeline')
                    raise
        except KeyboardInterrupt:
                self.hdmi_out.close()
                self.hdmi_in.close()
                print('close pipeline')
                raise

    def judge(self, img_slice):
        if((img_slice==255).all()):
            self.space.write(0)
#             print('not jump')
        else:
            self.space.write(1)
            print('jump')

In [ ]:
dino = DinoAgent()

In [ ]:
dino.video_pipeline()